# Notebook 4 - Naive-Bayes and Logistics Regression in NLP

## 7. Naive-Bayes Classifier

As stated in the previous notebook, Naive-Bayes is a supervised learning probabilistic classifier. It is based on applying Bayes' probability theorem and using the fact that occurrence of an event impacts the probability of another event. But how exactly does it work?

### 7.1. NB Fundamentals
The general purpose of classifiers is to *classify* samples from the dataset into 2 or more **classes**. Since we want to classify text, instead of term *sample* we will use term **document**. Thus, classifiers' task is to take an input document *d* and out of all possible classes, return a class *c*, to which the document *d* belongs.

Now, since NB is the probabilistic classifier, its role would be to **maximize the probability** of the predicted class c given the input document d.
<div style="text-align:center"><img src="res/eq1.png" alt="!!!equation 4.1!!!" width="300"/></div>

The intuition of Bayesian classification is to use **Bayes’ rule** to transform the equation above into their probabilities that have some useful properties.
<div style="text-align:center"><img src="res/eq2.png" alt="!!!equation 4.2!!!" width="300"/></div>

We then substitute the first equation into the second one to get:
<div style="text-align:center"><img src="res/eq3.png" alt="!!!equation 4.3!!!" width="400"/></div>

We can conveniently simplify the above equation by dropping the denominator *P(d)*. This is possible because we will be computing *P(d|c)P(c) / P(d)* for each possible class, but *P(d)* does not change for each class; we are always asking the most likely class for the same article *d*, which must have the same probability *P(d)*. Thus, we can choose the class that maximises the simpler formula

<div style="text-align:center"><img src="res/eq4.png" alt="!!!equation 4.4!!!" width="400"/></div>

Okay, but how do we actually represent a document *d*? We can represent a document as a set of **features** `d = (f1, f2, f3 ... fn)`. One way to define these features is to use the Bag-of-words model introduced in the Notebook 2. After contstructing the BOW of the complete dataset, we will be able to express each document as a vector of word counts. Thus, we can treat each vector value associated with a different word as a separate feature giving us information on the words (and optionally their counts) used in the document. Here we also introduce the first of two **simplifying assumptions**: since we use BOW, the **word ordering doesn't matter**. We don't care about the position of a word in a document.
<div style="text-align:center"><img src="res/eq5.png" alt="!!!equation 4.6!!!" width="400"/></div>

However, calculating `P(f1, f2, f3 ... fn | c)` requires computing all possible combinations of features (if BOW uses sum pooling than even more!). We need another simplifying assumption called the **naive Bayes assumption** - the conditional independence between features given the same class. Hence, we can multiply probabilites as follows:
<div style="text-align:center"><img src="res/eq6.png" alt="!!!equation 4.7!!!" width="400"/></div>

Resulting in a final equation:
<div style="text-align:center"><img src="res/eq8.png" alt="!!!equation 4.8!!!" width="400"/></div>


### 7.2 NB Training

So how do we train the classifier? How does it learn what actually is *P(c)* and *P(f|c)*? Starting with the first probability, we can simply use frequencies and derive it from the probability definition: the probability of a class in the dataset is the number of documents of this class divided be the total number of all documents. 
<div style="text-align:center"><img src="res/eq8,5.png" alt="!!!equation 4.11!!!" width="200"/></div>

Learning the probability of a features given a class *P(f<sub>i</sub>|c)* isn't more complicated. We assume a feature is just the existence of a word in the document’s bag of words (set of the vocabulary *V*), and so we’ll want *P(w|c)*, which we compute as **fraction of times the word w<sub>i</sub> appears among all words in all documents of topic c**.
<div style="text-align:center"><img src="res/eq9.png" alt="!!!equation 4.12!!!" width="400"/></div>

Let's consider the following example:

This is our training data:

| **Text** | **Lables** |
|----------|-----------|
|"What a great match" | sports |
|"The election results will be out tomorrow"| not sports |
|"The match was very boring"| sports |
|"It was a close election"| not sports |

To make the example easier to follow, let’s assume we applied some pre-processing to the sentences and removed stopwords. The resulting sentences are:

| **Text** | **Lables** |
|----------|------------|
|"great match"|  sports |
|"election results tomorrow"| not sports |
|"match boring"| sports |
|"close election"| not sports |

In our small corpus, we have 2 classes each having 2 senteces. Hence, the probability of each class is:

        P("sports") = 2/4 = 0.5
        P("not sports") = 2/4 = 0.5

Total unique features (words) for "sports": 3
Total unique features (words) for "not sports": 4

Let's say we want to assign a class to the following sentence "that was a very close, great match". After stop word removal it is "**close great match**". Now we need to perform some calculations:

1. Likelihood P("close great match"|sports) = P("close"|sports) * P("great"|sports) * P("match"|sports)
2. Likelihood P("close great match"|not sports) = P("close"|not sports) * P("great"|not sports) * P("match"|not sports)

| word | P(word\|sports) | P(word\|not sports)|
|------|----------------|-------------------|
| close | 0/3 | 1/4 |
| great | 1/3| 0/4|
| match | 2/3| 0/4|

We suspect that the correct class is "sport", right? Let's see what happens with the likelihood:
P("close great match"|sports) = 0/3 * 1/3 * 2/3
P("close great match"|sports) = 0

Oops... This example shows a very common situation - there were no training documents classified as "sports" containing word "close". As a result, P("close"|"sports") results in a painful zero, which also makes the product of probabilities equals 0. We can solve this issue using **smoothing**.

### 7.4 NB Smoothing & Unknown words


#### Laplace smoothing

Smoothing is used to avoid a situation that the classifier assigns zero probability to the whole document (as we can see above). This happens when a classifier sees a word, which IS present in the vocabulary (perhaps in a document of a different class), but it wasn't used in the given context. The intuition behind the smoothing is that we don't want classifier to assign zero probabilities to previously unseen events - the fact that something wasn't present in the training data, doesn't guarantee that it is impossible.

There are many smoothing algorithms but the simplest one is called **Laplace smoothing** or **Add-one smoothing**. The part of the classification algorithm which casues a problem is the probability of a features given a class *P(f<sub>i</sub>|c)*, which we interpret as the **fraction of times the word w<sub>i</sub> appears among all words in all documents of topic c**. If the numerator is equal 0, the whole probability is also equal 0. Laplace smoothing adds 1 to each count resulting in a new formula for the probability. Note that since we artificially increment the number of occurences of each word in the numerator and denominator, we basically add the size of the vocabulary |V| to the denominator. This results in the equation:

<div style="text-align:center"><img src="res/eq10.png" alt="!!!equation 4.14!!!" width="600"/></div>

#### Add-k smoothing

Add-one smoothing is not the only solution, and very often not the best one. Remember, that the probability mass is finite, which means that if we add some probability to one event, we have to remove it (preferably uniformly) from other events. Adding 1 to each word count, sometimes results in moving too much probability mass from rarely seen to totally unseen events. How can we deal with this situation? Well, the simplest solution is to add a smaller number than 1 to each word count (we don't care about the "integerity" of the word count anymore). 

This smoothing is called **Add-k smoothing** since it adds **k** to each word count. This, however, requires estimating what is the best value for **k**. It can be very different for different datasets and applications, so it should be adjusted to the problem.

#### Unseen words

How about previously unseed words in none of contexts (classes)? For example what happens if we try to test the classifier above on a document "*close funny match*"? Well, since we use the bag-of-words model, we have already seen (in Notebook 2) that this issue is unsolvable in a simple way, because we would have to modify all one-hot vectors. The only reasonable solution in this case is to **remove all previously unseen words**. Because of this, NB classifiers need rather big training datasets to perform well.

### 7.5 Completed NB example & Implementation

Ok, so let's complete our example using the Laplace smoothing.

The size of our vocabulary |V| = 7, so we will add it to all denominators.

| word | P(word\|sports) | P(word\|not sports)|
|------|----------------|-------------------|
| close | 1/10 | 2/11 |
| great | 2/10| 1/11|
| match | 3/10| 1/11|

P("close great match"|sports) = 1/10 * 2/10 * 3/10 = 0.006
P("close great match"|not sports) = 2/11 * 1/11 * 1/11 = 0.0015

Now, we have to multiply each probability by the probability of a class (posterior = likelihood * prior), which results in:
P("close great match"|sports)*P(sports) = 0.006 * 0.5 = 0.003
P("close great match"|not sports)*P(not sports) = 0.0015 * 0.5 = 0.00075

Thus, there is a higher probability that the test document is indeed about sports and this would be the decision of the NB classifier.

Now, let's implement exactly the same example using `Python` and `scikit-learn`! Firstly let's create training set and a test document.

In [ ]:
# 1 - sports, 0 - not sports
training_corpus = ["What a great match",
          "The election results will be out tomorrow",
          "The match was very boring",
          "It was a close election",
          ]
training_labels = [1, 0, 1, 0]

test_doc = "that was a very close, great match"

Now, we need to preprocess and vectorize documents to extract features (words).

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

from sklearn.feature_extraction.text import CountVectorizer

# Instantiate countvectorizer. You can sepcify what kind of preprocessing will be done by the CountVectorizer
# In this case we want all text in lowercase, remove stopwords, keep only alphanumeric characters.
count_vector = CountVectorizer(lowercase=True, stop_words='english', token_pattern='\w+')

# Fit training data
training_data = count_vector.fit_transform(training_corpus)
# Let's inspect vectors
print(count_vector.get_feature_names())
print(training_data.toarray())

# transform test data
test_doc_transform = count_vector.transform([test_doc])

After vectorizing, let's create and train the NB classifier. For numerical data we have used Gaussian NB classifier. For the NLP applications we will use Multinomial version of this classifier.

In [ ]:
# Create the classifier object and fit data. The classifier object is by default created with the Laplace smoothing
naive_bayes = MultinomialNB(alpha=1)  # alpha parameter specifies the k number from the add-k smoothing. 1 is the default value
naive_bayes.fit(training_data, training_labels)

# Make predictions.
predictions = naive_bayes.predict(test_doc_transform)
predictions

Result of 1, means the classifier predict that the test sentence was from the "sport" category. Now, let's try with this one:

In [ ]:
test_doc2 = "that was a very close, great election"
test_doc2_transform = count_vector.transform([test_doc2])
predictions = naive_bayes.predict(test_doc2_transform)
predictions

In [ ]:
# We can also inspect calculated probabilites.
print(naive_bayes.classes_)  # print the order of classes
print(naive_bayes.predict_proba(test_doc_transform))
print(naive_bayes.predict_proba(test_doc2_transform))


This was just a simple example to familiarize with the NB classifier. Let's look at a real word scenario!

### 7.6 News classification using NB

We’ll use a public dataset from the BBC comprised of 2225 articles, each labeled under one of 5 categories: business, entertainment, politics, sport or tech. Articles have been already made lowercase and cleaned from punctuation.
The goal is to train the classifier to predict what is the class of a given article. Let's start with loading the dataset.

In [ ]:
import pandas as pd
bbc_data = pd.read_csv("res/bbc-text.csv")
bbc_data = bbc_data[['category', 'text']]
bbc_data.head()

To develop a well working classifier it is important to know the structure of the dataset. Let's see what is the size of each class.

In [ ]:
bbc_data['category'].value_counts().plot.bar()

Okay, the dataset seems to be balanced. With this sort of data, imbalance in the dataset wouldn't be that problematic, but read this article if you are interestd in when imbalanced data can become a problem: https://medium.com/analytics-vidhya/what-is-balance-and-imbalance-dataset-89e8d7f46bc5

Now, let's vectorize documents and train the model!

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import (
    train_test_split, StratifiedShuffleSplit,
    cross_val_score)
from sklearn.naive_bayes import MultinomialNB

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    bbc_data['text'],
    bbc_data['category'],
    test_size=0.2,
    random_state=50,
)

#instantiate countvectorizer 
count_vector = CountVectorizer(lowercase=True, stop_words='english', token_pattern='\w+')

#fit training data
training_data = count_vector.fit_transform(X_train)

#transform test data
testing_data = count_vector.transform(X_test)

naive_bayes = MultinomialNB()
naive_bayes.fit(training_data,y_train)

predictions = naive_bayes.predict(testing_data)

In [ ]:
print('accuracy: {}'.format(accuracy_score(y_test, predictions)))
print('f1: {}'.format(f1_score(y_test, predictions, average='macro')))

Wow - thats pretty good. But maybe we were lucky and got an "easy" 20% test set. Let's use cross validation to exclude that possibility

In [ ]:
X_whole = count_vector.fit_transform(X_train)
# CV in sklearn needs lables in numerical values, so let's use LabelEncoder
le = LabelEncoder()
y_enc = le.fit_transform(y_train)
scores = cross_val_score(naive_bayes, X_whole, y_enc, cv=StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=100), scoring='accuracy')  # other scoring metric: f1_macro, f1_micro
# Mean accuracy scoring 
scores.mean()

This looks like a pretty decent score! Let's manually see how it works:

In [ ]:
tech_article = X_train[0]
business_article = X_train[1]

In [ ]:
tech_article[:100]

In [ ]:
business_article[:100]

Let's see what is the predicted class for the `tech_article`

In [ ]:
print(naive_bayes.predict(count_vector.transform([tech_article])))

In [ ]:
# And let's inspect probabilities assigned to all of the classes
dict(zip(naive_bayes.classes_, *naive_bayes.predict_proba(count_vector.transform([tech_article]))))

How about the `business_article`?

In [42]:
print(naive_bayes.predict(count_vector.transform([business_article])))

['business']


In [43]:
# And let's inspect probabilities assigned to all of the classes
dict(zip(naive_bayes.classes_, *naive_bayes.predict_proba(count_vector.transform([business_article]))))

{'business': 1.0,
 'entertainment': 1.8608928735341935e-89,
 'politics': 7.83023971567479e-69,
 'sport': 1.7129276449338016e-105,
 'tech': 3.1934048837492243e-86}

## 8. Logistic Regression Classifier

Logistic Regression is the another supervised learning algorithm and probabilistic classifier. This means that it will be also interested in calculating the probability *P(c|d)*:

<div style="text-align:center"><img src="res/eq1.png" alt="!!!equation 4.1!!!" width="300"/></div>

NB classifier used the Bayes' theorem and additional assumptions to derive another equation from the one above. It was interested in the probability of a document as a set of features given a class *P(d|c)*, which was learnt from the training dataset. 

Logistic regression does it in a different way. As a discriminative classifie, it tries to learn differences between classes rather than how class representatives "look like". Thus, it will directly try to learn probability of a class given a set of features representing a document *P(c|d)*. The key of this algorithm is in determining which features discrimanate documents in the most efficient way, by assigning these features appropriate **weights** (parameters). For examaple, in the "sports" or "not sports" text classification, the word "football" will probably
have strong positive weight and the word "princess", probably strong negative weight. 

In supervised machine learning, we have input features and sets of labels. To make predictions based on data, we use a function F with some parameters Θ to map features to output labels. To get an optimum mapping from features to labels, we have to minimise the cost function, which works by comparing how closely the output Ŷ is to the true labels Y from the training data. Learning takes place by updating the parameters and repeating the process until the cost is minimised.

<div style="text-align:center"><img src="res/sup_learning.jpeg" alt="!!!equation 4.1!!!" width="600"/></div>

Part of the Logistic Regression algorithm:
1. Feature representation - in our case using one-hot vectors, but in general using **word embeddings** (Notebook 5)
2. A classification function that gives the estimated class using *P(c|d)* - we will use two most popular ones - **sigmoid** and **softmax**
3. The cost function (or loss function) used for learning, which tells us the difference between the estimated class ĉ and the true class c. We will use **cross-entropy loss function**
4. An algorithm for optimizing the classifier by adjusting parameters and consequently minimizing the cost function. We will use **stochastic gradient decent**



Since we represent features using a vector `x=[x`<sub>`1`</sub>`, x`<sub>`2`</sub>`, x`<sub>`3`</sub>` ... x`<sub>`n`</sub>`]` and we need to associate some weights with each of these features, we can also represent these weights using a **weights vector** `w=[w`<sub>`1`</sub>`, w`<sub>`2`</sub>`, w`<sub>`3`</sub>` ... w`<sub>`n`</sub>`]`. Then, we can calculate the score for the test document by multiplying each feature with the associated weight and adding all results together. There is also a another parameter - the **bias term**, which is added to the sum.

<div style="text-align:center"><img src="res/eq11.png" alt="!!!equation 5.2!!!" width="300"/></div>

In other words, we want the sum of the **dot product** of the features and weights vectors and the bias term:

<div style="text-align:center"><img src="res/eq12.png" alt="!!!equation 5.3!!!" width="300"/></div>

But how do we map this score to the probability, since the score can be any real number from −∞ to ∞? We will use a **sigmoid function** *σ(z)*, which is also called the logistic function.

<div style="text-align:center"><img src="res/eq13.png" alt="!!!equation 5.3!!!" width="700"/></div>

As you can see, it is symmetric with respect to the point (0, 0.5) and it maps all real numbers to the range (0,1) - this is exactly what we need! For two classes (binomial logistic regression), class 0 and class 1, we have probabilities as follows:

<div style="text-align:center"><img src="res/eq14.png" alt="!!!equation 5.3!!!" width="300"/></div>



Okay, so how does the classifier learn the correct weights vector and the bias? It starts with some arbitrary values, which are then used to classify a training set. Each result is compared with the true label using the loss functions, in our case with the cross-entropy loss function. This function tells the classifier what is the difference between the classifier output and the true label. Taking into account loss, the classifier needs to adjust its parameters *θ* (in our case *θ = w, b*) to minimize the loss. This is equivalent to finding parameters *θ*, that minimize the loss function (find its minimum). 

Since the loss function is parameterized by weights and the bias term, finding its minimum is a multidimensional task. The popular algorithm for finding the minimum of a function is the Stochastic Gradient Decent (SGD). Gradient Decent is a method that finds a minimum of a function by figuring out in which direction (in the space of the parameters *θ*) the function’s slope is rising the most steeply, and moving in the opposite direction. The intuition is that if you are hiking in a canyon and trying to descend most quickly down to the river at the bottom, you might look around yourself 360 degrees, find the direction where the ground is sloping the steepest, and walk downhill in that direction.


I strongly encourage you to read [this chapter](https://web.stanford.edu/~jurafsky/slp3/5.pdf) of the book “Speech and Language Processing” by Daniel Jurafsky and James H. Martin as it explains the idea behind SGD very well.



**TODO**

- TODO maybe add some graphics/equations
- TODO add regularization (why we need this, norms)
